In [4]:
# input: LAS file, output csv

import pandas as pd
import os
from pathlib import Path

# How to use

1. Edit the variables in the cell below.
2. Run all cells by clicking "Cell > Run All" or "Ctrl + Enter" on each cell.


In [10]:
# edit these to point to your .las file
TEMPLATES_PARENT_FOLDER = "C:\\Users\\paigeplan\\Documents\\laser_settings_csv_maker\\data\\"
TEMPLATE_FOLDER_NAME = "#6 PLS_660 Material Settings"

OUTPUT_FOLDER = "C:\\Users\\paigeplan\\Documents\\laser_settings_csv_maker\\data\\"
OUTPUT_FILENAME = "Laser6_all_settings.csv"


# do not edit this line
create_machine_csv(TEMPLATES_PARENT_FOLDER + TEMPLATE_FOLDER_NAME + "\\", OUTPUT_FOLDER, OUTPUT_FILENAME)

IGNORING file: Basswood
IGNORING file: Plywood


In [7]:
def parse_file(filepath, title):
    # returns dataframe 
    
    colors = ["Black", "Red", "Green", "Yellow", "Blue", "Magenta", "Cyan", "Orange"]
    modes = ["Skip", "Rast", "Vect", "Vect", "Vect", "Vect", "Vect", "Vect"]
    
    laser_name = ""
    material_powers = []
    material_speeds = []
    material_ppi = []

    with open(filepath) as input_data:
    
        laser_name = input_data.readline().strip()[1:-1] # the last part takes off first and last brackets
        for line in input_data:
        
            current_line = line.split("=")
            if (current_line[0] == "PenPowers"):
                material_powers = current_line[1].strip().split(" ")
                material_powers = [int(x)/10 for x in material_powers]

            if (current_line[0] == "PenSpeedsCips"):
                material_speeds = current_line[1].strip().split(" ")
                material_speeds = [int(x)/10 for x in material_speeds]
            if (current_line[0] == "PenRates"):
                material_ppi = current_line[1].strip().split(" ")
                material_ppi = [int(x) for x in material_ppi]
    
    
    data = {laser_name:colors, "Mode": modes, "Power": material_powers, "Speed": material_speeds, "PPI": material_ppi}
    df = pd.DataFrame(data)
    df.set_index(laser_name, inplace=True) # removes the row numbers added by default
    
    return df



In [8]:
def create_machine_csv(input_folder, output_folder, output_filename):
    data_frames = []
    output_path = Path(OUTPUT_FOLDER, output_filename)
    output_path.unlink(missing_ok=True) #delete if already exists
    output_path.parent.mkdir(parents=True, exist_ok=True)
    pd.DataFrame().to_csv(output_path, mode="a")
    
    material_folders = [input_folder + file for file in os.listdir(input_folder)]
    for material_folder in material_folders:
        
        # add foldername row
        hacky_title_df = pd.DataFrame({" ": [material_folder]})
        hacky_title_df.set_index(" ", inplace=True) # removes the row numbers added by default
        hacky_title_df.to_csv(output_path, mode="a")
        
        # process files inside folders
        files = os.listdir(material_folder)
        for material_file in files:
            if Path(material_file).suffix != ".las": #ignore wrong filetypes
                print("IGNORING file: " + material_file)
            else:
                hacky_title_df = pd.DataFrame({" ": [material_file]})
                hacky_title_df.set_index(" ", inplace=True) # removes the row numbers added by default
                material_df = parse_file(material_folder + "\\" + material_file, material_file)
                hacky_title_df.to_csv(output_path, mode="a")
                material_df.to_csv(output_path, mode="a")
            